In [ ]:
# Python Version: "3.10.5"
# Java Version: "1.8.0_421"

In [ ]:
import subprocess

try:
    # Esegue il comando per ottenere la versione di Java
    version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
    print(version.decode('utf-8'))
except FileNotFoundError:
    # Gestisce il caso in cui Java non sia installato
    print("Java non è installato sul sistema. Si prega di installare Java 8 o successiva")
except subprocess.CalledProcessError as e:
    # Gestisce errori legati all'esecuzione del comando
    print(f"Errore durante l'esecuzione del comando Java: {e}")

In [ ]:
import os

print(os.getenv("JAVA_HOME"))
print(os.getenv("HADOOP_HOME"))

In [ ]:
#Commentare se si importa il file in env anaconda
%pip install -r ../../requirements.txt

In [ ]:
import sparknlp

# create or get Spark Session
#spark = sparknlp.start()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.4.1") \
    .getOrCreate()

In [ ]:
# Verifica della versione di Spark NLP
print("Spark NLP version:", sparknlp.version())
print("Apache Spark version:", spark.version)

In [ ]:
import sys
import os

# Aggiungi il percorso del modulo al PYTHONPATH
sys.path.append(os.path.abspath('../../Common'))

import UtilityNLP as nlpUtils
import UtilityClustering as cltUtils

In [ ]:
#Lettura Dataframe
df=spark.read.option("header",True).csv('../dati/input/esempio_frasi_1.csv')
df.cache()
df.show()

In [ ]:
#Definizione della pipeline per il Sentence Embedding con BERT
model = nlpUtils.nlp_pipeline_bert_sentence_embedding("descrizione").fit(df)
result_bert = model.transform(df)

In [ ]:
#Caching della Sentence Embedding
result_bert.cache()
result_bert.display()

In [ ]:
print("# dataset originale: {}".format(df.count()))
print("# dataset nuovo: {}".format(result_bert.count()))

In [ ]:
#Converte gli embedding delle sentences in colonne separate nel DataFrame 
# (ogni vettore N-dimensionale viene tramutato in un dataframe di N colonne)
result_df_exp=nlpUtils.convert_sentence_embedding_in_col(result_bert,["idcase","descrizione","dataapertura"])
result_df_exp.display()

In [ ]:
#Applichiamo la PCA sulle features ottenute dal sentence embedding (explode dell'embedding)
col_features=result_df_exp.columns[4:]
result_df_exp_filled = result_df_exp.dropna()
result, pca_model, loadings=cltUtils.pipelineStandardPCA(result_df_exp_filled, col_features, 30)

In [ ]:
#Plottiamo la varianza mantenuta in base al numero di PCA da scegliere
cltUtils.cumulativePCwithVariance(pca_model)

In [ ]:
#Plot della Silhuette per decidere il corretto valore K di cluster
cltUtils.silhouetteClusteringKMeans(result,"pca_features",m=2,n=20,i=2)

In [ ]:
#Applicazione del clustering
predictions_cluster_final, final_model=cltUtils.defineClustering(result, 12)

In [ ]:
predictions_cluster_final.select("idcase","descrizione","dataapertura","sentence","prediction").distinct().display()

In [ ]:
# Plot in 3D interattivo delle 3 PC con varianza maggiore (PC1 - PC2 - PC3) 
cltUtils.plotPCA3DInterattivo(predictions_cluster_final.where("dataapertura >= '2024-06-01'"), 
                              features='pca_features', predictions='prediction', additional_column='descrizione')

In [ ]:
# Plot in 3D interattivo delle 3 PC con varianza maggiore (PC1 - PC2 - PC3) marcate per cluster associato
# I punti dello spazio associati allo stesso cluster hanno lo stesso colore associato 
cltUtils.plotClustering3DInterattivo(predictions_cluster_final.where("dataapertura >= '2024-06-01'"), 
                                     features='pca_features', predictions='prediction', additional_column='descrizione')

In [ ]:
#Applicazione della TF-IDF per ogni cluster:
# Restituisce le prime N parole con il peso TF-IDF più alto per ciascuna predizione
docs_per_topic = predictions_cluster_final.groupby('prediction').agg(concat_ws(' ', collect_list(col("sentence"))).alias('Doc'))

topN=nlpUtils.top_n_words(docs_per_topic, inputCol="Doc", outputCol="features", ngram=3, N=10, targetCol="prediction")

topN.display()

In [ ]:
docs_per_topic = predictions_cluster_final.groupby('prediction').agg(concat_ws(' ', collect_list(col("sentence"))).alias('Doc'))

topN_4gram=nlpUtils.top_n_words(docs_per_topic, inputCol="Doc", outputCol="features", ngram=4, N=10, targetCol="prediction")

topN_4gram.display()

In [ ]:
predictions_cluster_final.groupBy("prediction").count().display()

In [ ]:
#Creazione del dataset finale con frase in linguaggio naturale e cluster associato
predictions_cluster_final.select("idcase",
    "descrizione","prediction").join(topN,"prediction","left").display()